In [23]:
import numpy as np
from scipy.signal import convolve2d
import rasterio
import os
from pathlib import Path

This notebook calculates neighborhood averages of the prepared datasets.

Pixels in a circle 2,5 and 10 kilometer distance around each pixel are put into zone 2,3 and 4. Zone 1 is the pixel itself. 
The zones are exclusive, so zone 3 is only the pixels between 2 and 5 km distance. 

In [22]:
from rural_beauty.config import CLC_coverage_EU_dir, DEM_EU_range, windpower_EU_raster, streets_EU_raster

In [32]:
os.listdir(CLC_coverage_EU_dir)

['code_heide.tif',
 'code_natgru.tif',
 'code_noveg.tif',
 'code_obst.tif',
 'code_seemee.tif',
 'code_spfr.tif',
 'code_stoer.tif',
 'code_wald.tif',
 'neighborhood']

In [10]:
zone_dict = {
    DEM_EU_range                           : ["zone1", "zone2", "zone3","zone1_2", "zone3_4"], 
    CLC_coverage_EU_dir / "code_obst.tif"  : ["zone1_4"], 
    CLC_coverage_EU_dir / "code_wald.tif"  : ["zone1_4"],
    CLC_coverage_EU_dir / "code_natgru.tif": ["zone2", "zone1_2"], 
    CLC_coverage_EU_dir / "code_acker.tif" : ["zone1_4", "zone1_2"], 
    CLC_coverage_EU_dir / "code_stoer.tif" : ["zone1_2", "zone2", "zone3"], 
    CLC_coverage_EU_dir / "code_noveg.tif" : ["zone2"], 
    windpower_EU_raster                    : ["zone1_4"], 
    streets_EU_raster                      : ["zone1_2"], 

}

In [21]:
import numpy as np

def create_circular_disc(circle_radius: int, overall_size: int) -> np.ndarray:
    # Create a 0/1 array of size 2*overall_size + 1 with an approximate circle of 1s in the middle, with radius circle_radius
    
    size = 2 * overall_size + 1  # Compute the matrix size based on the overall size
    disc = np.zeros((size, size), dtype=int)  # Initialize a zero matrix

    center = overall_size  # The center of the disc
    for y in range(size):
        for x in range(size):
            # Calculate the Euclidean distance from the center
            if np.sqrt((x - center) ** 2 + (y - center) ** 2) <= circle_radius:
                disc[y, x] = 1

    return disc

# Example usage
print(create_circular_disc(4, 4))


array([[0, 0, 0, 0, 1, 0, 0, 0, 0],
       [0, 0, 1, 1, 1, 1, 1, 0, 0],
       [0, 1, 1, 1, 1, 1, 1, 1, 0],
       [0, 1, 1, 1, 1, 1, 1, 1, 0],
       [1, 1, 1, 1, 1, 1, 1, 1, 1],
       [0, 1, 1, 1, 1, 1, 1, 1, 0],
       [0, 1, 1, 1, 1, 1, 1, 1, 0],
       [0, 0, 1, 1, 1, 1, 1, 0, 0],
       [0, 0, 0, 0, 1, 0, 0, 0, 0]])

In [20]:
def get_zone_mean(data:np.ndarray, na_value:float, kernel:np.ndarray) -> np.ndarray:
    #apply a filter with a given kernel to a 2d array and return the mean value for each element of the data array
    kernel = kernel.astype(float)
    kernel /= kernel.sum()

    data_filled = np.where(data == na_value, 0, data)
    # Perform the convolution
    convolved_sum = convolve2d(data_filled, kernel, mode='valid')
    return convolved_sum

In [25]:
def modify_filename(file_path:str, add_string:str, new_folder:str) -> str:
    # Split the filename and the extension
    base, extension = os.path.splitext(file_path)

    # Get directory and basename
    dir_name, base_name = os.path.split(base)

    # Add the string to the basename
    new_base_name = f"{base_name}_{add_string}"

    # Construct the new path
    new_full_path = os.path.join(dir_name, new_folder, new_base_name + extension)

    # Ensure the directory exists
    new_directory = os.path.dirname(new_full_path)
    os.makedirs(new_directory, exist_ok=True)

    return new_full_path

In [26]:
# Create the circular disc
def write_specific_zone_mean(input_dict:str) -> None:
    overall_radius = 10
    zone_kernels = {
    "zone4" : create_circular_disc(9.9, overall_radius) - create_circular_disc(4.9, overall_radius),
    "zone3" : create_circular_disc(4.9, 5) - create_circular_disc(1.9, 5),
    "zone2" : create_circular_disc(1.9, 2) - create_circular_disc(0.9, 2),
    "zone1_4" : create_circular_disc(9.9, overall_radius),
    "zone1_2" : create_circular_disc(1.9, 2),
    "zone3_4" : create_circular_disc(9.9, overall_radius) - create_circular_disc(1.9, overall_radius)
    }
    
    for input_path, zone_names in input_dict.items():
        
        with rasterio.open(input_path) as raster_in:
            for zone_name in zone_names:
                kernel = zone_kernels[zone_name]
                meta = raster_in.meta.copy()
                na_value = raster_in.nodata
                array_in = raster_in.read(1)  # Reading the first band
    
                # Define the output path within the new directory
                output_path = modify_filename(input_path, zone_name, 'neighborhood')

                if os.path.exists(output_path):
                    print(f"{output_path} already exists skipping...")
                    continue
    
                print(output_path)
                output_array = get_zone_mean(array_in, na_value, kernel)
        
                # Create a new raster file for output
                meta.update(dtype=rasterio.float32, count=1)  # Update meta if necessary
        
                with rasterio.open(output_path, 'w', **meta) as raster_out:
                    raster_out.write(output_array, 1)  # Write output array to the first band

In [12]:
write_specific_zone_mean(zone_dict)

/media/sf_Granular/beauty/data/cleaned/dem/neighborhood/DEM_DE_range_zone1_2.tif already exists skipping...
/media/sf_Granular/beauty/data/cleaned/dem/neighborhood/DEM_DE_range_zone3_4.tif already exists skipping...
/media/sf_Granular/beauty/data/cleaned/clc/layer_coverage_DE/neighborhood/code_obst_zone1_4.tif already exists skipping...
/media/sf_Granular/beauty/data/cleaned/clc/layer_coverage_DE/neighborhood/code_wald_zone1_4.tif already exists skipping...
/media/sf_Granular/beauty/data/cleaned/clc/layer_coverage_DE/neighborhood/code_natgru_zone2.tif already exists skipping...
/media/sf_Granular/beauty/data/cleaned/clc/layer_coverage_DE/neighborhood/code_acker_zone1_4.tif already exists skipping...
/media/sf_Granular/beauty/data/cleaned/clc/layer_coverage_DE/neighborhood/code_stoer_zone2.tif already exists skipping...
/media/sf_Granular/beauty/data/cleaned/clc/layer_coverage_DE/neighborhood/code_stoer_zone3.tif already exists skipping...
/media/sf_Granular/beauty/data/cleaned/clc/laye